# Trabajo Práctico Final: Linear/Quadratic Discriminant Analysis (LDA/QDA)

### Definición: Clasificador Bayesiano

Sean $k$ poblaciones, $x \in \mathbb{R}^p$ puede pertenecer a cualquiera $g \in \mathcal{G}$ de ellas. Bajo un esquema bayesiano, se define entonces $\pi_j \doteq P(G = j)$ la probabilidad *a priori* de que $X$ pertenezca a la clase *j*, y se **asume conocida** la distribución condicional de cada observable dado su clase $f_j \doteq f_{X|G=j}$.

De esta manera dicha probabilidad *a posteriori* resulta
$$
P(G|_{X=x} = j) = \frac{f_{X|G=j}(x) \cdot p_G(j)}{f_X(x)} \propto f_j(x) \cdot \pi_j
$$

La regla de decisión de Bayes es entonces
$$
H(x) \doteq \arg \max_{g \in \mathcal{G}} \{ P(G|_{X=x} = j) \} = \arg \max_{g \in \mathcal{G}} \{ f_j(x) \cdot \pi_j \}
$$

es decir, se predice a $x$ como perteneciente a la población $j$ cuya probabilidad a posteriori es máxima.

*Ojo, a no desesperar! $\pi_j$ no es otra cosa que una constante prefijada, y $f_j$ es, en su esencia, un campo escalar de $x$ a simplemente evaluar.*

### Distribución condicional

Para los clasificadores de discriminante cuadrático y lineal (QDA/LDA) se asume que $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma_j)$, es decir, se asume que cada población sigue una distribución normal.

Por definición, se tiene entonces que para una clase $j$:
$$
f_j(x) = \frac{1}{(2 \pi)^\frac{p}{2} \cdot |\Sigma_j|^\frac{1}{2}} e^{- \frac{1}{2}(x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j)}
$$

Aplicando logaritmo (que al ser una función estrictamente creciente no afecta el cálculo de máximos/mínimos), queda algo mucho más práctico de trabajar:

$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$

Observar que en este caso $C=-\frac{p}{2} \log(2\pi)$, pero no se tiene en cuenta ya que al tener una constante aditiva en todas las clases, no afecta al cálculo del máximo.

### LDA

En el caso de LDA se hace una suposición extra, que es $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma)$, es decir que las poblaciones no sólo siguen una distribución normal sino que son de igual matriz de covarianzas. Reemplazando arriba se obtiene entonces:

$$
\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$

Ahora, como $-\frac{1}{2}\log |\Sigma|$ es común a todas las clases se puede incorporar a la constante aditiva y, distribuyendo y reagrupando términos sobre $(x-\mu_j)^T \Sigma^{-1} (x- \mu_j)$ se obtiene finalmente:

$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$

### Entrenamiento/Ajuste

Obsérvese que para ambos modelos, ajustarlos a los datos implica estimar los parámetros $(\mu_j, \Sigma_j) \; \forall j = 1, \dots, k$ en el caso de QDA, y $(\mu_j, \Sigma)$ para LDA.

Estos parámetros se estiman por máxima verosimilitud, de manera que los estimadores resultan:

* $\hat{\mu}_j = \bar{x}_j$ el promedio de los $x$ de la clase *j*
* $\hat{\Sigma}_j = s^2_j$ la matriz de covarianzas estimada para cada clase *j*
* $\hat{\pi}_j = f_{R_j} = \frac{n_j}{n}$ la frecuencia relativa de la clase *j* en la muestra
* $\hat{\Sigma} = \frac{1}{n} \sum_{j=1}^k n_j \cdot s^2_j$ el promedio ponderado (por frecs. relativas) de las matrices de covarianzas de todas las clases. *Observar que se utiliza el estimador de MV y no el insesgado*

Es importante notar que si bien todos los $\mu, \Sigma$ deben ser estimados, la distribución *a priori* puede no inferirse de los datos sino asumirse previamente, utilizándose como entrada del modelo.

### Predicción

Para estos modelos, al igual que para cualquier clasificador Bayesiano del tipo antes visto, la estimación de la clase es por método *plug-in* sobre la regla de decisión $H(x)$, es decir devolver la clase que maximiza $\hat{f}_j(x) \cdot \hat{\pi}_j$, o lo que es lo mismo $\log\hat{f}_j(x) + \log\hat{\pi}_j$.

## Estructura del código

## Modelo

In [1]:
import numpy as np
from numpy.linalg import det, inv

In [2]:
class ClassEncoder:
  def fit(self, y):
    self.names = np.unique(y)
    self.name_to_class = {name:idx for idx, name in enumerate(self.names)}
    self.fmt = y.dtype
    # Q1: por que no hace falta definir un class_to_name para el mapeo inverso?

  def _map_reshape(self, f, arr):
    return np.array([f(elem) for elem in arr.flatten()]).reshape(arr.shape)
    # Q2: por que hace falta un reshape?

  def transform(self, y):
    return self._map_reshape(lambda name: self.name_to_class[name], y)

  def fit_transform(self, y):
    self.fit(y)
    return self.transform(y)

  def detransform(self, y_hat):
    return self._map_reshape(lambda idx: self.names[idx], y_hat)

In [3]:
class BaseBayesianClassifier:
  def __init__(self):
    self.encoder = ClassEncoder()

  def _estimate_a_priori(self, y):
    a_priori = np.bincount(y.flatten().astype(int)) / y.size
    # Q3: para que sirve bincount?
    return np.log(a_priori)

  def _fit_params(self, X, y):
    # estimate all needed parameters for given model
    raise NotImplementedError()

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    raise NotImplementedError()

  def fit(self, X, y, a_priori=None):
    # first encode the classes
    y = self.encoder.fit_transform(y)

    # if it's needed, estimate a priori probabilities
    self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)

    # check that a_priori has the correct number of classes
    assert len(self.log_a_priori) == len(self.encoder.names), "A priori probabilities do not match number of classes"

    # now that everything else is in place, estimate all needed parameters for given model
    self._fit_params(X, y)
    # Q4: por que el _fit_params va al final? no se puede mover a, por ejemplo, antes de la priori?

  def predict(self, X):
    # this is actually an individual prediction encased in a for-loop
    m_obs = X.shape[1]
    y_hat = np.empty(m_obs, dtype=self.encoder.fmt)

    for i in range(m_obs):
      encoded_y_hat_i = self._predict_one(X[:,i].reshape(-1,1))
      y_hat[i] = self.encoder.names[encoded_y_hat_i]

    # return prediction as a row vector (matching y)
    return y_hat.reshape(1,-1)

  def _predict_one(self, x):
    # calculate all log posteriori probabilities (actually, +C)
    log_posteriori = [ log_a_priori_i + self._predict_log_conditional(x, idx) for idx, log_a_priori_i
                  in enumerate(self.log_a_priori) ]

    # return the class that has maximum a posteriori probability
    return np.argmax(log_posteriori)

In [4]:
class QDA(BaseBayesianClassifier):

  def _fit_params(self, X, y):
    # estimate each covariance matrix
    self.inv_covs = [inv(np.cov(X[:,y.flatten()==idx], bias=True))
                      for idx in range(len(self.log_a_priori))]
    # Q5: por que hace falta el flatten y no se puede directamente X[:,y==idx]?
    # Q6: por que se usa bias=True en vez del default bias=False?
    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]
    # Q7: que hace axis=1? por que no axis=0?

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_covs[class_idx]
    unbiased_x =  x - self.means[class_idx]
    return 0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x

In [5]:
class TensorizedQDA(QDA):

    def _fit_params(self, X, y):
        # ask plain QDA to fit params
        super()._fit_params(X,y)

        # stack onto new dimension
        self.tensor_inv_cov = np.stack(self.inv_covs)
        self.tensor_means = np.stack(self.means)

    def _predict_log_conditionals(self,x):
        unbiased_x = x - self.tensor_means
        inner_prod = unbiased_x.transpose(0,2,1) @ self.tensor_inv_cov @ unbiased_x

        return 0.5*np.log(det(self.tensor_inv_cov)) - 0.5 * inner_prod.flatten()

    def _predict_one(self, x):
        # return the class that has maximum a posteriori probability
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x))

## Código para pruebas

Seteamos los datos

In [6]:
# hiperparámetros
rng_seed = 6543

In [7]:
from sklearn.datasets import load_iris, fetch_openml

def get_iris_dataset():
  data = load_iris()
  X_full = data.data
  y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
  return X_full, y_full

def get_penguins():
    # get data
    df, tgt = fetch_openml(name="penguins", return_X_y=True, as_frame=True)

    # drop non-numeric columns
    df.drop(columns=["island","sex"], inplace=True)

    # drop rows with missing values
    mask = df.isna().sum(axis=1) == 0
    df = df[mask]
    tgt = tgt[mask]

    return df.values, tgt.to_numpy().reshape(-1,1)

# showing for iris
X_full, y_full = get_iris_dataset()

print(f"X: {X_full.shape}, Y:{y_full.shape}")

X: (150, 4), Y:(150, 1)


In [8]:
# peek data matrix
X_full[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [9]:
# peek target vector
y_full[:5]

array([['setosa'],
       ['setosa'],
       ['setosa'],
       ['setosa'],
       ['setosa']], dtype='<U10')

Separamos el dataset en train y test para medir performance

In [10]:
# preparing data, train - test validation
# 70-30 split
from sklearn.model_selection import train_test_split

def split_transpose(X, y, test_sz, random_state):
    # split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_sz, random_state=random_state)

    # transpose so observations are column vectors
    return X_train.T, y_train.T, X_test.T, y_test.T

def accuracy(y_true, y_pred):
  return (y_true == y_pred).mean()

train_x, train_y, test_x, test_y = split_transpose(X_full, y_full, 0.3, rng_seed)

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(4, 105) (1, 105) (4, 45) (1, 45)


Entrenamos un QDA y medimos su accuracy

In [11]:
qda = QDA()
qda.fit(train_x, train_y)

In [12]:
train_acc = accuracy(train_y, qda.predict(train_x))
test_acc = accuracy(test_y, qda.predict(test_x))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0190 while test error is 0.0000


Con el magic %%timeit podemos estimar el tiempo que tarda en correr una celda en base a varias ejecuciones. Por poner un ejemplo, acá vamos a estimar lo que tarda un ciclo completo de QDA y también su inferencia (predicción).

Ojo! a veces [puede ser necesario ejecutarlo varias veces](https://stackoverflow.com/questions/10994405/python-timeit-results-cached-instead-of-calculated) para obtener resultados consistentes.

In [13]:
%%timeit
qda.predict(test_x)

1.69 ms ± 148 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%%timeit

model = QDA()
model.fit(train_x, train_y)
model.predict(test_x)

3.44 ms ± 91.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Consigna

## Implementación
1. Entrenar un modelo QDA sobre el dataset *iris* utilizando las distribuciones *a priori* a continuación ¿Se observan diferencias?¿Por qué cree? _Pista: comparar con las distribuciones del dataset completo, **sin splitear**_.
    1. Uniforme (cada clase tiene probabilidad 1/3)
    2. Una clase con probabilidad 0.9, las demás 0.05 (probar las 3 combinaciones)
2. Repetir el punto anterior para el dataset *penguin*.
3. Implementar el modelo LDA, entrenarlo y testearlo contra los mismos sets que QDA ¿Se observan diferencias? ¿Podría decirse que alguno de los dos es notoriamente mejor que el otro?
4. Utilizar otros 2 (dos) valores de *random seed* para obtener distintos splits de train y test, y repetir la comparación del punto anterior ¿Las conclusiones previas se mantienen?
5. Estimar y comparar los tiempos de predicción de las clases `QDA` y `TensorizedQDA`. De haber diferencias ¿Cuáles pueden ser las causas?


**Sugerencia:** puede resultar de utilidad para los puntos de comparación utilizar tablas del siguiente estilo:

<center>

Modelo | Dataset | Seed | Error (train) | Error (test)
:---: | :---: | :---: | :---: | :---:
QDA | Iris | 125 | 0.55 | 0.85
LDA | Iris | 125 | 0.22 | 0.8

</center>

## Preguntas teóricas

1. En LDA se menciona que la función a maximizar puede ser, mediante operaciones, convertida en:
$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$
Mostrar los pasos por los cuales se llega a dicha expresión.
2. Explicar, utilizando las respectivas funciones a maximizar, por qué QDA y LDA son "quadratic" y "linear".
3. La implementación de QDA estima la probabilidad condicional utilizando `0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x` que no es *exactamente* lo descrito en el apartado teórico ¿Cuáles son las diferencias y por qué son expresiones equivalentes?

El espíritu de esta componente práctica es la de establecer un mínimo de trabajo aceptable para su entrega; se invita al alumno a explorar otros aspectos que generen curiosidad, sin sentirse de ninguna manera limitado por la consigna.

## Ejercicio teórico

Sea una red neuronal de dos capas, la primera de 3 neuronas y la segunda de 1 con los parámetros inicializados con los siguientes valores:
$$
w^{(1)} =
\begin{pmatrix}
0.1 & -0.5 \\
-0.3 & -0.9 \\
0.8 & 0.02
\end{pmatrix},
b^{(1)} = \begin{pmatrix}
0.1 \\
0.5 \\
0.8
\end{pmatrix},
w^{(2)} =
\begin{pmatrix}
-0.4 & 0.2 & -0.5
\end{pmatrix},
b^{(2)} = 0.7
$$

y donde cada capa calcula su salida vía

$$
y^{(i)} = \sigma (w^{(i)} \cdot x^{(i)}+b^{(i)})
$$

donde $\sigma (z) = \frac{1}{1+e^{-z}}$ es la función sigmoidea .

\\
Dada la observación $x=\begin{pmatrix}
1.8 \\
-3.4
\end{pmatrix}$, $y=5$ y la función de costo $J(\theta)=\frac{1}{2}(\hat{y}_\theta-y)^2$, calcular las derivadas de J respecto de cada parámetro $w^{(1)}$, $w^{(2)}$, $b^{(1)}$, $b^{(2)}$.

*Nota: Con una sigmoidea a la salida jamás va a poder estimar el 5 "pedido", pero eso no afecta al mecanismo de backpropagation!*

## Preguntas en el código
Previamente las preguntas "técnicas" en comentarios en el código eran parte del TP, y buscaban que el alumno logre entrar en el detalle de por qué cada linea de código es como es y en el orden en el que está. Ya no forman parte de la consigna, pero se aconseja al alumno intentar responderlas. Las respuestas a las mismas se encuentran en un archivo separado.

## Opcional

### QDA

Debido a la forma cuadrática de QDA, no se puede predecir para *n* observaciones en una sola pasada (utilizar $X \in \mathbb{R}^{p \times n}$ en vez de $x \in \mathbb{R}^p$) sin pasar por una matriz de *n x n* en donde se computan todas las interacciones entre observaciones. Se puede acceder al resultado recuperando sólo la diagonal de dicha matriz, pero resulta ineficiente en tiempo y (especialmente) en memoria. Aún así, es *posible* que el modelo funcione más rápido.

1. Implementar el modelo `FasterQDA` (se recomienda heredarlo de TensorizedQDA) de manera de eliminar el ciclo for en el método predict.
2. Comparar los tiempos de predicción de `FasterQDA` con `TensorizedQDA` y `QDA`
3. Mostrar (puede ser con un print) dónde aparece la mencionada matriz de *n x n*, donde *n* es la cantidad de observaciones a predecir.
4.Demostrar
$$
diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)
$$ es decir, que se puede "esquivar" la matriz de *n x n* usando matrices de *n x p*.
5.Utilizar la propiedad antes demostrada para reimplementar la predicción del modelo `FasterQDA` de forma eficiente. ¿Hay cambios en los tiempos de predicción?


### LDA

1. "Tensorizar" el modelo LDA y comparar sus tiempos de predicción con el modelo antes implementado. *Notar que, en modo tensorizado, se puede directamente precomputar $\mu^T \cdot \Sigma^{-1} \in \mathbb{R}^{k \times 1 \times p}$ y guardar eso en vez de $\Sigma^{-1}$.*
2. LDA no sufre del problema antes descrito de QDA debido a que no computa productos internos, por lo que no tiene un verdadero costo extra en memoria predecir "en batch". Implementar el modelo `FasterLDA` y comparar sus tiempos de predicción con las versiones anteriores de LDA.

# Desarrollo:

1. Entrenar un modelo QDA sobre el dataset *iris* utilizando las distribuciones *a priori* a continuación ¿Se observan diferencias?¿Por qué cree? _Pista: comparar con las distribuciones del dataset completo, **sin splitear**_.
    1. Uniforme (cada clase tiene probabilidad 1/3)
    2. Una clase con probabilidad 0.9, las demás 0.05 (probar las 3 combinaciones)

El punto 1 se realizó de dos formas. En primera instancia spliteando los datos y en el segundo caso considerando los datasets completos.

Método 1 (Splitted)

In [15]:
import pandas as pd
#CREAMOS DATA FRAME PARA ALMACENAR LOS DATOS:
df_results = pd.DataFrame(index=[], columns=["Modelo",	"Dataset",	"Seed",	"Error (train)","Error (test)"])

In [16]:
#distribuciones a priori uniforme:
priori_prob = np.array([1/3, 1/3, 1/3])

qda_uniform = QDA()

qda_uniform.fit(train_x, train_y, priori_prob)

train_acc = accuracy(train_y, qda_uniform.predict(train_x))
test_acc = accuracy(test_y, qda_uniform.predict(test_x))

In [17]:
#guardamos datos en el dataframe:
df_results.at[0, "Modelo"] = "QDAUniform"
df_results.at[0, "Dataset"] = "Iris"
df_results.at[0, "Seed"] = rng_seed
df_results.at[0, "Error (train)"] = 1-train_acc
df_results.at[0, "Error (test)"] = 1-test_acc

In [18]:
#Una clase con probabilidad 0.9, las demás 0.05 (probar las 3 combinaciones)
#primer combinacion:
priori_prob = np.array([0.9, 0.05, 0.05])

qda_differentprob = QDA()

qda_differentprob.fit(train_x, train_y, priori_prob)

train_acc = accuracy(train_y, qda_differentprob.predict(train_x))
test_acc = accuracy(test_y, qda_differentprob.predict(test_x))


df_results.at[1, "Modelo"] = "QDA with [0.9, 0.05, 0.05]"
df_results.at[1, "Dataset"] = "Iris"
df_results.at[1, "Seed"] = rng_seed
df_results.at[1, "Error (train)"] = 1-train_acc
df_results.at[1, "Error (test)"] = 1-test_acc

#segunda combinacion:
priori_prob = np.array([0.05, 0.9, 0.05])

qda_differentprob = QDA()

qda_differentprob.fit(train_x, train_y, priori_prob)

train_acc = accuracy(train_y, qda_differentprob.predict(train_x))
test_acc = accuracy(test_y, qda_differentprob.predict(test_x))


df_results.at[2, "Modelo"] = "QDA with [0.05, 0.9, 0.05]"
df_results.at[2, "Dataset"] = "Iris"
df_results.at[2, "Seed"] = rng_seed
df_results.at[2, "Error (train)"] = 1-train_acc
df_results.at[2, "Error (test)"] = 1-test_acc

#tercera combinacion:
priori_prob = np.array([0.05, 0.05, 0.9])

qda_differentprob = QDA()

qda_differentprob.fit(train_x, train_y, priori_prob)

train_acc = accuracy(train_y, qda_differentprob.predict(train_x))
test_acc = accuracy(test_y, qda_differentprob.predict(test_x))


df_results.at[3, "Modelo"] = "QDA with [0.05, 0.05, 0.9]"
df_results.at[3, "Dataset"] = "Iris"
df_results.at[3, "Seed"] = rng_seed
df_results.at[3, "Error (train)"] = 1-train_acc
df_results.at[3, "Error (test)"] = 1-test_acc

print(df_results)

                       Modelo Dataset  Seed Error (train) Error (test)
0                  QDAUniform    Iris  6543      0.028571          0.0
1  QDA with [0.9, 0.05, 0.05]    Iris  6543      0.028571          0.0
2  QDA with [0.05, 0.9, 0.05]    Iris  6543      0.047619     0.022222
3  QDA with [0.05, 0.05, 0.9]    Iris  6543      0.038095     0.044444


Método 2 (Full)

In [19]:
X_full, y_full = get_iris_dataset()

classes = np.unique(y_full)
print(classes)

# Entrenamos el modelo QDA con distribución uniforme (probabilidades iguales)
qda_uniform = QDA()
prior_uniform = np.array([1/3, 1/3, 1/3])
qda_uniform.fit(X_full.T, y_full.T, prior_uniform)

# Entrenamos el modelo QDA con una clase con alta probabilidad y las demás bajas

# Clase 0 con alta probabilidad (0.9), las otras dos con baja (0.05)
qda_high_prob_class_0 = QDA()
prior_high_prob_class_0 = np.array([0.9, 0.05, 0.05])
qda_high_prob_class_0.fit(X_full.T, y_full.T, prior_high_prob_class_0)

# Clase 1 con alta probabilidad (0.9), las otras dos con baja (0.05)
qda_high_prob_class_1 = QDA()
prior_high_prob_class_1 = np.array([0.05, 0.9, 0.05])
qda_high_prob_class_1.fit(X_full.T, y_full.T, prior_high_prob_class_1)

# Clase 2 con alta probabilidad (0.9), las otras dos con baja (0.05)
qda_high_prob_class_2 = QDA()
prior_high_prob_class_2 = np.array([0.05, 0.05, 0.9])
qda_high_prob_class_2.fit(X_full.T, y_full.T, prior_high_prob_class_2)

# Realizamos las predicciones de los modelos
predicted_uniform = qda_uniform.predict(X_full.T)
predicted_high_prob_class_0 = qda_high_prob_class_0.predict(X_full.T)
predicted_high_prob_class_1 = qda_high_prob_class_1.predict(X_full.T)
predicted_high_prob_class_2 = qda_high_prob_class_2.predict(X_full.T)

# Calculamos el accuracy para comparar las distribuciones
accuracy_uniform = accuracy(y_full.T, predicted_uniform)
accuracy_high_prob_class_0 = accuracy(y_full.T, predicted_high_prob_class_0)
accuracy_high_prob_class_1 = accuracy(y_full.T, predicted_high_prob_class_1)
accuracy_high_prob_class_2 = accuracy(y_full.T, predicted_high_prob_class_2)

print(f"Uniform (apparent) error is : {1 - accuracy_uniform:.4f}")
print(classes[0] + f" (apparent) error is : {1 - accuracy_high_prob_class_0:.4f}")
print(classes[1] + f" (apparent) error is : {1 - accuracy_high_prob_class_1:.4f}")
print(classes[2] + f" (apparent) error is : {1 - accuracy_high_prob_class_2:.4f}")

['setosa' 'versicolor' 'virginica']
Uniform (apparent) error is : 0.0200
setosa (apparent) error is : 0.0200
versicolor (apparent) error is : 0.0333
virginica (apparent) error is : 0.0400


Las diferencias observadas se debe al enfoque del modelo hacia ciertas clases según las distribuciones a priori, lo que influye en su capacidad para clasificar correctamente las muestras, priorizando ciertas clases sobre otras.

# 


#

2. Repetir el punto anterior para el dataset *penguin*.

In [20]:
#pedimos los datos a penguins:
X_penguins_full, y_penguins_full = get_penguins()

C:\Users\Windows\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\datasets\_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [21]:
#transponemos datos:
penguins_train_x, penguins_train_y, penguins_test_x, penguins_test_y = split_transpose(X_full, y_full, 0.4, rng_seed)

In [22]:
#caso uniforme:
priori_prob = np.array([1/3, 1/3, 1/3])

qda_uniform_penguins = QDA()

qda_uniform_penguins.fit(penguins_train_x, penguins_train_y, priori_prob)

penguins_train_acc = accuracy(penguins_train_y, qda_uniform.predict(penguins_train_x))
penguins_test_acc = accuracy(penguins_test_y, qda_uniform.predict(penguins_test_x))

df_results.at[4, "Modelo"] = "QDAUniform"
df_results.at[4, "Dataset"] = "Penguins"
df_results.at[4, "Seed"] = rng_seed
df_results.at[4, "Error (train)"] = 1-penguins_train_acc
df_results.at[4, "Error (test)"] = 1-penguins_test_acc

In [23]:
#Una clase con probabilidad 0.9, las demás 0.05 (probar las 3 combinaciones)
#primer combinacion:
priori_prob = np.array([0.9, 0.05, 0.05])

qda_penguins = QDA()

qda_penguins.fit(penguins_train_x, penguins_train_y, priori_prob)

penguins_train_acc = accuracy(penguins_train_y, qda_penguins.predict(penguins_train_x))
penguins_test_acc = accuracy(penguins_test_y, qda_penguins.predict(penguins_test_x))

df_results.at[5, "Modelo"] = "QDA with [0.9, 0.05, 0.05]"
df_results.at[5, "Dataset"] = "Penguins"
df_results.at[5, "Seed"] = rng_seed
df_results.at[5, "Error (train)"] = 1-penguins_train_acc
df_results.at[5, "Error (test)"] = 1-penguins_test_acc

#segunda combinacion:
priori_prob = np.array([0.05, 0.9, 0.05])

qda_penguins = QDA()

qda_penguins.fit(penguins_train_x, penguins_train_y, priori_prob)

penguins_train_acc = accuracy(penguins_train_y, qda_penguins.predict(penguins_train_x))
penguins_test_acc = accuracy(penguins_test_y, qda_penguins.predict(penguins_test_x))

df_results.at[6, "Modelo"] = "QDA with [0.05, 0.9, 0.05]"
df_results.at[6, "Dataset"] = "Penguins"
df_results.at[6, "Seed"] = rng_seed
df_results.at[6, "Error (train)"] = 1-penguins_train_acc
df_results.at[6, "Error (test)"] = 1-penguins_test_acc

#tercera combinacion:
priori_prob = np.array([0.05, 0.05, 0.9])

qda_penguins = QDA()

qda_penguins.fit(penguins_train_x, penguins_train_y, priori_prob)

penguins_train_acc = accuracy(penguins_train_y, qda_penguins.predict(penguins_train_x))
penguins_test_acc = accuracy(penguins_test_y, qda_penguins.predict(penguins_test_x))

df_results.at[7, "Modelo"] = "QDA with [0.05, 0.05, 0.9]"
df_results.at[7, "Dataset"] = "Penguins"
df_results.at[7, "Seed"] = rng_seed
df_results.at[7, "Error (train)"] = 1-penguins_train_acc
df_results.at[7, "Error (test)"] = 1-penguins_test_acc

print(df_results)

                       Modelo   Dataset  Seed Error (train) Error (test)
0                  QDAUniform      Iris  6543      0.028571          0.0
1  QDA with [0.9, 0.05, 0.05]      Iris  6543      0.028571          0.0
2  QDA with [0.05, 0.9, 0.05]      Iris  6543      0.047619     0.022222
3  QDA with [0.05, 0.05, 0.9]      Iris  6543      0.038095     0.044444
4                  QDAUniform  Penguins  6543      0.022222     0.016667
5  QDA with [0.9, 0.05, 0.05]  Penguins  6543      0.022222     0.016667
6  QDA with [0.05, 0.9, 0.05]  Penguins  6543      0.033333          0.0
7  QDA with [0.05, 0.05, 0.9]  Penguins  6543      0.033333         0.05


Método 2 (Full)

In [24]:
X_full, y_full = get_penguins()

classes = np.unique(y_full)
print("Classes: "+classes)

# Entrenamos el modelo QDA con distribución uniforme (probabilidades iguales)
qda_uniform = QDA()
prior_uniform = np.array([1/3, 1/3, 1/3])
qda_uniform.fit(X_full.T, y_full.T, prior_uniform)

# Entrenamos el modelo QDA con una clase con alta probabilidad y las demás bajas

# Clase 0 con alta probabilidad (0.9), las otras dos con baja (0.05)
qda_high_prob_class_0 = QDA()
prior_high_prob_class_0 = np.array([0.9, 0.05, 0.05])
qda_high_prob_class_0.fit(X_full.T, y_full.T, prior_high_prob_class_0)

# Clase 1 con alta probabilidad (0.9), las otras dos con baja (0.05)
qda_high_prob_class_1 = QDA()
prior_high_prob_class_1 = np.array([0.05, 0.9, 0.05])
qda_high_prob_class_1.fit(X_full.T, y_full.T, prior_high_prob_class_1)

# Clase 2 con alta probabilidad (0.9), las otras dos con baja (0.05)
qda_high_prob_class_2 = QDA()
prior_high_prob_class_2 = np.array([0.05, 0.05, 0.9])
qda_high_prob_class_2.fit(X_full.T, y_full.T, prior_high_prob_class_2)

# Realizamos las predicciones de los modelos
predicted_uniform = qda_uniform.predict(X_full.T)
predicted_high_prob_class_0 = qda_high_prob_class_0.predict(X_full.T)
predicted_high_prob_class_1 = qda_high_prob_class_1.predict(X_full.T)
predicted_high_prob_class_2 = qda_high_prob_class_2.predict(X_full.T)

# Calculamos el accuracy para comparar las distribuciones
accuracy_uniform = accuracy(y_full.T, predicted_uniform)
accuracy_high_prob_class_0 = accuracy(y_full.T, predicted_high_prob_class_0)
accuracy_high_prob_class_1 = accuracy(y_full.T, predicted_high_prob_class_1)
accuracy_high_prob_class_2 = accuracy(y_full.T, predicted_high_prob_class_2)

print(f"Uniform (apparent) error is : {1 - accuracy_uniform:.4f}")
print(classes[0] + f" (apparent) error is : {1 - accuracy_high_prob_class_0:.4f}")
print(classes[1] + f" (apparent) error is : {1 - accuracy_high_prob_class_1:.4f}")
print(classes[2] + f" (apparent) error is : {1 - accuracy_high_prob_class_2:.4f}")

['Classes: Adelie' 'Classes: Chinstrap' 'Classes: Gentoo']
Uniform (apparent) error is : 0.0088
Adelie (apparent) error is : 0.0175
Chinstrap (apparent) error is : 0.0351
Gentoo (apparent) error is : 0.0088


C:\Users\Windows\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\datasets\_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Se aprecia nuevamente que el sesgo producido en las distribucionea a priori modifica la capacidad de predicción del clasificador afectando los resultados.

#


#

3. Implementar el modelo LDA, entrenarlo y testearlo contra los mismos sets que QDA ¿Se observan diferencias? ¿Podría decirse que alguno de los dos es notoriamente mejor que el otro?

In [25]:
#CREAMOS LA FUNCIÓN LDA:
#MODELO LDA:
class LDA(BaseBayesianClassifier):

  def _fit_params(self, X, y):
    #se necesita la matriz de covarianza inversa, primero calculamos la matriz de covarianza para cada clase y luego el promedio ponderado de estas matrices, por ultimo su inversa.
    self.covs = [np.cov(X[:,y.flatten()==idx], bias=True) for idx in range(len(self.log_a_priori))]
    #idx tiene los indices de las clases.
    #se toman todas las filas de X para cuando "y" coincida con idx (o sea, cada subconjunto de datos que se agarra estan asociados a una clase)
    #con "inv" calculamos la inversa de la matriz
    # Q5: por que hace falta el flatten y no se puede directamente X[:,y==idx]?
    #PROMEDIO PONDERADO:
    self.pond_cov = np.zeros((self.covs[0].shape[0], self.covs[0].shape[1]))
    unique_y = np.unique(y)
    for i in range(len(unique_y)):
      #numero de muestras para cada clase:
      n_j = np.sum(y == unique_y[i])
      self.pond_cov += n_j * self.covs[i]

    self.inv_pond_cov = inv( self.pond_cov / len(y) )


    # Q6: por que se usa bias=True en vez del default bias=False? 
    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]
    # Q7: que hace axis=1? por que no axis=0?

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    unbiased_x =  x -0.5 * self.means[class_idx]
    return self.means[class_idx].T @ self.inv_pond_cov @ unbiased_x
    #aca tenemos la distribución condicional de cada observable dado su clase, como se dijo en el paper es un campo escalar de x


In [26]:
#Todo lo anterior pero con LDA:

#caso uniforme
priori_prob = np.array([1/3, 1/3, 1/3])

lda_uniform = LDA()

lda_uniform.fit(train_x, train_y, priori_prob)

lda_train_acc = accuracy(train_y, lda_uniform.predict(train_x))
lda_test_acc = accuracy(test_y, lda_uniform.predict(test_x))

df_results.at[8, "Modelo"] = "LDAUniform"
df_results.at[8, "Dataset"] = "Iris"
df_results.at[8, "Seed"] = rng_seed
df_results.at[8, "Error (train)"] = 1-lda_train_acc
df_results.at[8, "Error (test)"] = 1-lda_test_acc




In [27]:
#Una clase con probabilidad 0.9, las demás 0.05 (probar las 3 combinaciones)
#primer combinacion:
priori_prob = np.array([0.9, 0.05, 0.05])

lda_differentprob = LDA()

lda_differentprob.fit(train_x, train_y, priori_prob)

lda_train_acc = accuracy(train_y, lda_differentprob.predict(train_x))
lda_test_acc = accuracy(test_y, lda_differentprob.predict(test_x))

df_results.at[9, "Modelo"] = "LDA with [0.9, 0.05, 0.05]"
df_results.at[9, "Dataset"] = "Iris"
df_results.at[9, "Seed"] = rng_seed
df_results.at[9, "Error (train)"] = 1-lda_train_acc
df_results.at[9, "Error (test)"] = 1-lda_test_acc

#segunda combinacion:
priori_prob = np.array([0.05, 0.9, 0.05])

lda_differentprob = LDA()

lda_differentprob.fit(train_x, train_y, priori_prob)

lda_train_acc = accuracy(train_y, lda_differentprob.predict(train_x))
lda_test_acc = accuracy(test_y, lda_differentprob.predict(test_x))

df_results.at[10, "Modelo"] = "LDA with [0.05, 0.9, 0.05]"
df_results.at[10, "Dataset"] = "Iris"
df_results.at[10, "Seed"] = rng_seed
df_results.at[10, "Error (train)"] = 1-lda_train_acc
df_results.at[10, "Error (test)"] = 1-lda_test_acc


#tercera combinacion:
priori_prob = np.array([0.05, 0.05, 0.9])

lda_differentprob = LDA()

lda_differentprob.fit(train_x, train_y, priori_prob)

lda_train_acc = accuracy(train_y, lda_differentprob.predict(train_x))
lda_test_acc = accuracy(test_y, lda_differentprob.predict(test_x))

df_results.at[11, "Modelo"] = "LDA with [0.05, 0.05, 0.9]"
df_results.at[11, "Dataset"] = "Iris"
df_results.at[11, "Seed"] = rng_seed
df_results.at[11, "Error (train)"] = 1-lda_train_acc
df_results.at[11, "Error (test)"] = 1-lda_test_acc




In [35]:
#LDA PARA PINGUINS:
lda_uniform_penguins = LDA()

lda_uniform_penguins.fit(penguins_train_x, penguins_train_y)

penguins_train_acc = accuracy(penguins_train_y, qda_uniform.predict(penguins_train_x))
penguins_test_acc = accuracy(penguins_test_y, qda_uniform.predict(penguins_test_x))
print(f"Train (apparent) error is {1-penguins_train_acc:.4f} while test error is {1-penguins_test_acc:.4f}")

df_results.at[12, "Modelo"] = "LDAUniform"
df_results.at[12, "Dataset"] = "Penguins"
df_results.at[12, "Seed"] = rng_seed
df_results.at[12, "Error (train)"] = 1-penguins_train_acc
df_results.at[12, "Error (test)"] = 1-penguins_test_acc



Train (apparent) error is 1.0000 while test error is 1.0000


In [29]:
#Una clase con probabilidad 0.9, las demás 0.05 (probar las 3 combinaciones)
#primer combinacion:
priori_prob = np.array([0.9, 0.05, 0.05])

lda_differentprob_penguins = LDA()

lda_differentprob_penguins.fit(penguins_train_x, penguins_train_y, priori_prob)

penguins_train_acc = accuracy(penguins_train_y, lda_differentprob_penguins.predict(penguins_train_x))
penguins_test_acc = accuracy(penguins_test_y, lda_differentprob_penguins.predict(penguins_test_x))

df_results.at[13, "Modelo"] = "LDA with [0.9, 0.05, 0.05]"
df_results.at[13, "Dataset"] = "Penguins"
df_results.at[13, "Seed"] = rng_seed
df_results.at[13, "Error (train)"] = 1-penguins_train_acc
df_results.at[13, "Error (test)"] = 1-penguins_test_acc

#segunda combinacion:
priori_prob = np.array([0.05, 0.9, 0.05])

lda_differentprob_penguins = LDA()

lda_differentprob_penguins.fit(penguins_train_x, penguins_train_y, priori_prob)

penguins_train_acc = accuracy(penguins_train_y, lda_differentprob_penguins.predict(penguins_train_x))
penguins_test_acc = accuracy(penguins_test_y, lda_differentprob_penguins.predict(penguins_test_x))

df_results.at[14, "Modelo"] = "LDA with [0.05, 0.9, 0.05]"
df_results.at[14, "Dataset"] = "Penguins"
df_results.at[14, "Seed"] = rng_seed
df_results.at[14, "Error (train)"] = 1-penguins_train_acc
df_results.at[14, "Error (test)"] = 1-penguins_test_acc

#tercera combinacion:
priori_prob = np.array([0.05, 0.05, 0.9])

lda_differentprob_penguins = LDA()

lda_differentprob_penguins.fit(penguins_train_x, penguins_train_y, priori_prob)

penguins_train_acc = accuracy(penguins_train_y, lda_differentprob_penguins.predict(penguins_train_x))
penguins_test_acc = accuracy(penguins_test_y, lda_differentprob_penguins.predict(penguins_test_x))

df_results.at[15, "Modelo"] = "LDA with [0.05, 0.05, 0.9]"
df_results.at[15, "Dataset"] = "Penguins"
df_results.at[15, "Seed"] = rng_seed
df_results.at[15, "Error (train)"] = 1-penguins_train_acc
df_results.at[15, "Error (test)"] = 1-penguins_test_acc

print(df_results)

                        Modelo   Dataset  Seed Error (train) Error (test)
0                   QDAUniform      Iris  6543      0.028571          0.0
1   QDA with [0.9, 0.05, 0.05]      Iris  6543      0.028571          0.0
2   QDA with [0.05, 0.9, 0.05]      Iris  6543      0.047619     0.022222
3   QDA with [0.05, 0.05, 0.9]      Iris  6543      0.038095     0.044444
4                   QDAUniform  Penguins  6543      0.022222     0.016667
5   QDA with [0.9, 0.05, 0.05]  Penguins  6543      0.022222     0.016667
6   QDA with [0.05, 0.9, 0.05]  Penguins  6543      0.033333          0.0
7   QDA with [0.05, 0.05, 0.9]  Penguins  6543      0.033333         0.05
8                   LDAUniform      Iris  6543      0.028571          0.0
9   LDA with [0.9, 0.05, 0.05]      Iris  6543      0.666667     0.666667
10  LDA with [0.05, 0.9, 0.05]      Iris  6543      0.695238          0.6
11  LDA with [0.05, 0.05, 0.9]      Iris  6543      0.638095     0.733333
12                  LDAUniform  Pengui

Como puede apreciarse, con QDA se obtiene una mejor predicción que con LDA.

#


#

4. Utilizar otros 2 (dos) valores de *random seed* para obtener distintos splits de train y test, y repetir la comparación del punto anterior ¿Las conclusiones previas se mantienen?

Seed = 1

In [30]:
# hiperparámetros
rng_seed = 1

# showing for iris
X_full, y_full = get_iris_dataset()

print(f"X: {X_full.shape}, Y:{y_full.shape}")

qda = QDA()
qda.fit(train_x, train_y)

lda = LDA()
lda.fit(train_x, train_y)

# preparing data, train - test validation
# 70-30 split

train_x, train_y, test_x, test_y = split_transpose(X_full, y_full, 0.3, rng_seed)

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

qda_iris_seed1_train_acc = accuracy(train_y, qda.predict(train_x))
qda_iris_seed1_test_acc = accuracy(test_y, qda.predict(test_x))

lda_iris_seed1_train_acc = accuracy(train_y, lda.predict(train_x))
lda_iris_seed1_test_acc = accuracy(test_y, lda.predict(test_x))
print("Iris results")
print(f"QDA train (apparent) error is {1 - qda_iris_seed1_train_acc:.4f} while test error is {1 - qda_iris_seed1_test_acc:.4f}")
print(f"LDA train (apparent) error is {1 - lda_iris_seed1_train_acc:.4f} while test error is {1 - lda_iris_seed1_test_acc:.4f}")

X: (150, 4), Y:(150, 1)
(4, 105) (1, 105) (4, 45) (1, 45)
Iris results
QDA train (apparent) error is 0.0190 while test error is 0.0000
LDA train (apparent) error is 0.3048 while test error is 0.4000


Seed = 2

In [31]:
# hiperparámetros
rng_seed = 2

# showing for iris
X_full, y_full = get_iris_dataset()

print(f"X: {X_full.shape}, Y:{y_full.shape}")

qda = QDA()
qda.fit(train_x, train_y)

lda = LDA()
lda.fit(train_x, train_y)

# preparing data, train - test validation
# 70-30 split

train_x, train_y, test_x, test_y = split_transpose(X_full, y_full, 0.3, rng_seed)

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

qda_iris_seed2_train_acc = accuracy(train_y, qda.predict(train_x))
qda_iris_seed2_test_acc = accuracy(test_y, qda.predict(test_x))

lda_iris_seed2_train_acc = accuracy(train_y, lda.predict(train_x))
lda_iris_seed2_test_acc = accuracy(test_y, lda.predict(test_x))
print("Iris results")
print(f"QDA train (apparent) error is {1 - qda_iris_seed2_train_acc:.4f} while test error is {1 - qda_iris_seed2_test_acc:.4f}")
print(f"LDA train (apparent) error is {1 - lda_iris_seed2_train_acc:.4f} while test error is {1 - lda_iris_seed2_test_acc:.4f}")

X: (150, 4), Y:(150, 1)
(4, 105) (1, 105) (4, 45) (1, 45)
Iris results
QDA train (apparent) error is 0.0190 while test error is 0.0000
LDA train (apparent) error is 0.3143 while test error is 0.3333


Sí, las conclusiones se mantienen

#


#

5. Estimar y comparar los tiempos de predicción de las clases `QDA` y `TensorizedQDA`. De haber diferencias ¿Cuáles pueden ser las causas?

In [32]:
%%timeit

model = QDA()
model.fit(train_x, train_y)
model.predict(test_x)

3.54 ms ± 222 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [33]:
%%timeit

model = TensorizedQDA()
model.fit(train_x, train_y)
model.predict(test_x)

2.56 ms ± 38.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


El rendimiento superior de TensorizedQDA en comparación con QDA probablemente se debe a la forma en que se están realizando los cálculos en cada uno de los métodos. TensorizedQDA aprovecha el uso de operaciones vectorizadas y manipulación de matrices, lo que generalmente mejora el rendimiento en comparación con el enfoque no matricial de QDA.